In [1]:
import numpy as np
import math
import scipy.special as sci
from matplotlib import pyplot as plt

#reservoir properties
phi_data = 0.088
h_data = 170
cf_data = 0
rw_data = 0.333
#re_data 

In [2]:
#import dataset from xlsx
import xlrd
#import col 0: shut-in time (t) 
#import col 1: pressure data (Pws)

N_data = 5039
N_data_q = N_data  #for this case
N_data_shutin = 13898
res_Excel = xlrd.open_workbook('114947_Pseudotime_PTA_Input.xlsx')

# read pressure form Excel file

t = np.zeros((N_data,1))
Pwf = np.zeros((N_data,1)) 
qo = np.zeros((N_data_q,1))
dt = np.zeros((N_data_shutin,1))
Pws = np.zeros((N_data_shutin,1)) 
    
for i in range (0,N_data):
    t[i] = res_Excel.sheet_by_name('pressure(history)').cell(i,0).value
    Pwf[i] = res_Excel.sheet_by_name('pressure(history)').cell(i,1).value
    qo[i] = res_Excel.sheet_by_name('rate').cell(i,1).value
for i in range (0,N_data_shutin):
    dt[i] = res_Excel.sheet_by_name('pressure(shut-in)').cell(i,0).value
    Pws[i] = res_Excel.sheet_by_name('pressure(shut-in)').cell(i,1).value

In [3]:
#use initial condition as reference
P_ref = 9330
T_ref = 300+459.67
Sg = 0.7

#gas fluid properties
import numpy as np
import math


Psc = 14.65
Tsc = 519.67
Mw_air = 28.97
R_const = 10.732

def z_factor(P,T,Sg):
    P_pr = P/P_critical(Sg)
    T_pr = T/T_critical(Sg)
    
    A1 = 0.3265         
    A2 = -1.07          
    A3 = -0.5339        
    A4 = 0.01569        
    A5 = -0.05165        
    A6 = 0.5475
    A7 = -0.7361       
    A8 = 0.1844         
    A9 = 0.1056         
    A10 = 0.6134        
    A11 = 0.721
        
    C1 = A1 + A2 / T_pr + A3 / (T_pr * T_pr * T_pr) + A4 / (T_pr * T_pr * T_pr * T_pr) + A5 / (T_pr * T_pr * T_pr * T_pr * T_pr)
    C2 = A6 + A7 / T_pr + A8 / (T_pr * T_pr)
    C3 = A9 * (A7 / T_pr + A8 / (T_pr * T_pr))

    Ze = 1
    for i in range(0,100):
        rho_pr = 0.27 * (P_pr / (Ze * T_pr))
        C4 = A11 * rho_pr * rho_pr
        Zc = 1 + C1 * rho_pr + C2 * rho_pr * rho_pr - C3 * (rho_pr * rho_pr * rho_pr * rho_pr * rho_pr) + A10 * (1 + C4) * (rho_pr * rho_pr / (T_pr * T_pr * T_pr)) * np.exp(-C4)
        if (np.abs(Ze - Zc) < 0.001):
            break
        else:
            fz = Ze - Zc
            dfdZ = 1 + (rho_pr * rho_pr / Ze) * (C1 / rho_pr + 2 * C2 - 5 * C3 * (rho_pr * rho_pr * rho_pr) + 2 * A10 * np.exp(-C4) * (1 + C4 - C4 * C4) / (T_pr * T_pr * T_pr))
            Ze = Ze - fz / dfdZ
    
    return Zc

def T_critical(Sg):  #Sutton
    return 169.2 + 349.5 * Sg - 74 * Sg * Sg 
def P_critical(Sg): #Sutton
    return 756.8 - 131 * Sg - 3.6 * Sg**2
def T_reduced(T,Sg): #T in Rankine
    return T/T_critical(Sg)
def P_reduced(P,Sg): #T in Rankine
    return P/P_critical(Sg)

def rho_gas(P, T, Sg):
    Mwg = Sg * Mw_air
    z = z_factor(P,T,Sg)
    return P*Mwg/z/R_const/T

def visc_gas(P,T,Sg):
    z = z_factor(P,T,Sg)
    gasdensPT = rho_gas(P, T, Sg)
    gasdensPT = gasdensPT * 1000 / (2.20462262184878 * ((0.3048 * 100) ** 3)) 
    Mwg = Sg * Mw_air
    XK = (9.379 + 0.01607 * Mwg) * ((T ** 1.5) / (209.2 + 19.26 * Mwg + T))
    X = 3.448 + 986.4 / T + 0.01009 * Mwg
    Y = 2.447 - 0.2224 * X
    return (XK / 10000) * np.exp(X * (gasdensPT ** Y))

def cg(P,T,Sg):
    dP = 1
    z_left2 = z_factor(P-2*dP,T,Sg)
    z_left1 = z_factor(P-dP,T,Sg)
    z_cen = z_factor(P,T,Sg)
    z_right1 = z_factor(P+dP,T,Sg)
    z_right2 = z_factor(P+2*dP,T,Sg)
    dzdP = (z_right1-z_left1)/2/dP
    #dzdP = (-z_left2+8*z_left1-8*z_right1+z_right2)/12/dP
    return 1/P-1/z_cen*dzdP
    
def Bg(P,T,Sg):
    Bg = Psc/Tsc*z_factor(P,T,Sg)*T/P
    return Bg/5.6146*1000

In [4]:
#calculate reference value for gas properties
visc_gas_ref = visc_gas(P_ref,T_ref,Sg)
Bg_ref = Bg(P_ref,T_ref,Sg)
zg_ref = z_factor(P_ref,T_ref,Sg)
cg_ref = cg(P_ref,T_ref,Sg)

print(visc_gas_ref)
print(Bg_ref)
print(zg_ref)
print(cg_ref)

0.03604856910532351
0.5483710452483469
1.3413435518599353
5.097542413316822e-05


In [5]:
G = 3.23e+9 #from RTA
#compute Gp
Gp = np.zeros((N_data,1))
Gp_sum = 0
for i in range(0,N_data):
    if i == 0:
        #Gp_sum += (t[i]-0)*qo[i]*1000 
            
        #use power-law integration trick for the first panel
        B_power_int = np.log(qo[i+1]/qo[i])/np.log(t[i+1]/t[i])
        A_power_int = qo[i]/t[i]**B_power_int
        Gp_sum += A_power_int/(B_power_int+1)*t[i]**(B_power_int+1)*1000  #t in Days, convert MSCF to SCF
        Gp[i] = Gp_sum
    else:
        Gp_sum += (t[i]-t[i-1])*(qo[i]+qo[i-1])/2*1000
        Gp[i] = Gp_sum

print(Gp)

[[1.50131776e+07]
 [2.50401776e+07]
 [3.44991776e+07]
 ...
 [2.38343749e+09]
 [2.38344594e+09]
 [2.38355484e+09]]


In [6]:
#Material balance Equation + p_avg
from scipy.optimize import fsolve

p_avg = np.zeros((N_data,1))
for i in range(0,N_data): 
    def p_avg_solve(z):
        p_avg = z[0]
        zg_avg = z_factor(p_avg,T_ref,Sg)

        F = np.empty((1))
        F[0] = p_avg/zg_avg - P_ref/zg_ref*(1-Gp[i]/G)
        return F

    zGuess = np.array([4500])
    p_avg[i] = fsolve(p_avg_solve,zGuess)

print(p_avg)

[[9246.57018605]
 [9185.68042922]
 [9128.77533735]
 ...
 [1715.84301901]
 [1715.82620086]
 [1715.60945651]]


In [7]:
#pseudotime calculation (ta)
#create table for pseudotime calculation
    #col 0 pressure
    #col 1 time
    #col 2 1_divide_visc_ct
    #col 3 P_divide_visc_ct_panel 
    #col 4 sum P_divide_visc_ct_panel 
    #col 5 t_a

N_pseudotime = N_data 
pseudotime_matrix = np.zeros((N_pseudotime,6))
sum_pseudotime_integral = 0
for i in range(0,N_pseudotime):
        pseudotime_matrix[i,0] = p_avg[i]
        pseudotime_matrix[i,1] = t[i]

for i in range(0,N_pseudotime):
    P_step = pseudotime_matrix[i,0]
    ct_step = cg(P_step,T_ref,Sg)+cf_data
    pseudotime_matrix[i,2] = 1/visc_gas(P_step,T_ref,Sg)/ct_step
    if i == 0:
        pseudotime_matrix[i,3] = 0
    else:
        pseudotime_matrix[i,3] = 1/2*(pseudotime_matrix[i,2]+pseudotime_matrix[i-1,2])*(pseudotime_matrix[i,1]-pseudotime_matrix[i-1,1])
    
    sum_pseudotime_integral += pseudotime_matrix[i,3]
    pseudotime_matrix[i,4] = sum_pseudotime_integral
    
    pseudotime_matrix[i,5] = pseudotime_matrix[i,4]*visc_gas_ref*cg_ref

ta = pseudotime_matrix[:,5]
print(ta)

[0.00000000e+00 1.00099749e+00 1.99317991e+00 ... 1.65810578e+03
 1.65828493e+03 1.65846406e+03]


In [8]:
#pseudo shut-in time calculation (dta)
#create table for pseudotime calculation
    #col 0 pressure
    #col 1 time
    #col 2 1_divide_visc_ct
    #col 3 P_divide_visc_ct_panel 
    #col 4 sum P_divide_visc_ct_panel 
    #col 5 dt_a

N_pseudo_shutin_time = N_data_shutin
print(N_pseudo_shutin_time)
pseudo_shutin_time_matrix = np.zeros((N_pseudo_shutin_time,6))
sum_pseudo_shutin_time_integral = 0
for i in range(0,N_pseudo_shutin_time):
    pseudo_shutin_time_matrix[i,0] = Pws[i]
    pseudo_shutin_time_matrix[i,1] = dt[i]

for i in range(0,N_pseudo_shutin_time):
    P_step = pseudo_shutin_time_matrix[i,0]
    ct_step = cg(P_step,T_ref,Sg)+cf_data
    pseudo_shutin_time_matrix[i,2] = 1/visc_gas(P_step,T_ref,Sg)/ct_step
    if i == 0:
        pseudo_shutin_time_matrix[i,3] = 0
    else:
        pseudo_shutin_time_matrix[i,3] = 1/2*(pseudo_shutin_time_matrix[i,2]+pseudo_shutin_time_matrix[i-1,2])*(pseudo_shutin_time_matrix[i,1]-pseudo_shutin_time_matrix[i-1,1])
    
    sum_pseudo_shutin_time_integral += pseudo_shutin_time_matrix[i,3]
    pseudo_shutin_time_matrix[i,4] = sum_pseudo_shutin_time_integral
    
    pseudo_shutin_time_matrix[i,5] = pseudo_shutin_time_matrix[i,4]*visc_gas_ref*cg_ref

dta = pseudo_shutin_time_matrix[:,5]
print(dta)

13898
[0.00000000e+00 6.74561860e-04 1.35612555e-03 ... 1.36183199e+02
 1.36222470e+02 1.36261702e+02]


In [9]:
#material-balance pseudotime calculation (ta_MBE)
#create table for pseudotime calculation
ta_MBE = np.zeros((N_pseudotime,1))
sum_ta_MBE = 0

for i in range(0,N_pseudotime):
    if i == 0:
        #sum_ta_MBE += (qo[i]+0)*(ta[i]-0)/2
        
        #use power-law integration trick for the first panel
        B_power_int = np.log(qo[i+1]/qo[i])/np.log(ta[i+1]/ta[i])
        A_power_int = qo[i]/ta[i]**B_power_int
        Gp_sum += A_power_int/(B_power_int+1)*ta[i]**(B_power_int+1)*1000  #t in Days, convert MSCF to SCF
        
        
        ta_MBE[i] = sum_ta_MBE/qo[i]
    else:
        sum_ta_MBE += (qo[i]+qo[i-1])*(ta[i]-ta[i-1])/2
        ta_MBE[i] = sum_ta_MBE/qo[i]

print(ta_MBE)

[[0.00000000e+00]
 [1.10393773e+00]
 [1.97659834e+00]
 ...
 [6.78439195e+04]
 [           inf]
 [5.26438576e+03]]


C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide


In [10]:
#create table for pseudopressure calculation
    #col 0 pressure
    #col 1 P_divide_visc_z
    #col 2 P_divide_visc_z_panel 
    #col 3 sum P_divide_visc_z_panel 
    #col 4 Pwf_a
#use prestep pressure

N_pseudopressure = 5000
pmax_pseudopressure = 10000
pseudopressure_matrix = np.zeros((N_pseudopressure,5))
sum_pseudopressure_integral = 0

for i in range(0,N_pseudopressure):
    pseudopressure_matrix[i,0] = pmax_pseudopressure/N_pseudopressure*i
    P_step = pseudopressure_matrix[i,0]
    pseudopressure_matrix[i,1] = P_step/visc_gas(P_step,T_ref,Sg)/z_factor(P_step,T_ref,Sg)
    if i == 0:
        pseudopressure_matrix[i,2] = 0
    else:
        pseudopressure_matrix[i,2] = 1/2*(pseudopressure_matrix[i,1]+pseudopressure_matrix[i-1,1])*(pseudopressure_matrix[i,0]-pseudopressure_matrix[i-1,0])
    
    sum_pseudopressure_integral += pseudopressure_matrix[i,2]
    pseudopressure_matrix[i,3] = sum_pseudopressure_integral
    
    pseudopressure_matrix[i,4] = pseudopressure_matrix[i,3]*visc_gas_ref*zg_ref/P_ref

Pwf_a_ref = pseudopressure_matrix[:,4]
Pwf_ref = pseudopressure_matrix[:,0] 

In [11]:
#interpolate pseudopressure
P_ref_a = np.interp(P_ref,Pwf_ref,Pwf_a_ref)
Pwf_a = np.interp(Pwf,Pwf_ref,Pwf_a_ref)
Pws_a = np.interp(Pws,Pwf_ref,Pwf_a_ref)

print(P_ref_a)
print(Pwf_a)
print(Pws_a)

7540.132771577818
[[244.80544898]
 [380.60005681]
 [378.34935445]
 ...
 [ 64.44441526]
 [147.75894853]
 [ 15.84020399]]
[[ 389.64902498]
 [ 399.98771672]
 [ 407.76221303]
 ...
 [2926.53887196]
 [2927.32316167]
 [2928.21155619]]


In [12]:
#export adjusted shut-in data
compare_matrix = np.zeros((N_data_shutin,2))
compare_matrix[:,0] = dta[:] 
compare_matrix[:,1] = Pws_a[:,0]
compare_matrix[0,0] = 0
for i in range(0,N_data_shutin):
    if np.isinf(compare_matrix[i,0]) == True:
        compare_matrix[i,0] = 0
    if  np.isinf(compare_matrix[i,1]) == True:
        compare_matrix[i,1] = 0

print(compare_matrix)

#export adjusted rate profile
compare_matrix_rate = np.zeros((N_data,2))
compare_matrix_rate[:,0] = ta[:] 
compare_matrix_rate[:,1] = qo[:,0]
compare_matrix_rate[0,0] = 0
for i in range(0,N_data):
    if np.isinf(compare_matrix_rate[i,0]) == True:
        compare_matrix_rate[i,0] = 0
    if  np.isinf(compare_matrix_rate[i,1]) == True:
        compare_matrix_rate[i,1] = 0

print(compare_matrix_rate)

#write file
import xlsxwriter
workbook = xlsxwriter.Workbook('114947 Pseudotime for PTA (output).xlsx')
worksheet = workbook.add_worksheet('pressure')
    
col = 0

for row, data in enumerate(compare_matrix):
    worksheet.write_row(row, col, data)
    
worksheet = workbook.add_worksheet('rate')
    
col = 0

for row, data in enumerate(compare_matrix_rate):
    worksheet.write_row(row, col, data)

workbook.close()

[[0.00000000e+00 3.89649025e+02]
 [6.74561860e-04 3.99987717e+02]
 [1.35612555e-03 4.07762213e+02]
 ...
 [1.36183199e+02 2.92653887e+03]
 [1.36222470e+02 2.92732316e+03]
 [1.36261702e+02 2.92821156e+03]]
[[0.00000000e+00 1.09620000e+04]
 [1.00099749e+00 9.09200000e+03]
 [1.99317991e+00 9.82600000e+03]
 ...
 [1.65810578e+03 1.68999996e+01]
 [1.65828493e+03 0.00000000e+00]
 [1.65846406e+03 2.17800003e+02]]
